In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset, Dataset, concatenate_datasets
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,BertForSequenceClassification,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml
from tqdm import tqdm
import logging
import json
import warnings
import random
import pandas as pd

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

CS_DATA_PATH = PATH + '/data/CS/processed/'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = 'fav-kky/FERNET-C5'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

BATCH_SIZE = 32

In [73]:
babe_train = load_dataset('csv',data_files = CS_DATA_PATH + 'BABE/train.csv')['train']
babe_test = load_dataset('csv',data_files = CS_DATA_PATH + 'BABE/test.csv')['train']
babe_all = load_dataset('csv',data_files = CS_DATA_PATH + 'BABE/babe.csv')['train']

PRETRAINED_PATH = '/home/horyctom/bias-detection-thesis/src/models/trained/all_balanced.pth'
FINAL_PATH = '/home/horyctom/bias-detection-thesis/src/models/trained/final_classifier.pth'
SUBJ_PATH = '/home/horyctom/bias-detection-thesis/src/models/trained/subj_balanced.pth'

## Prepare model

In [3]:
model_all = BertForSequenceClassification.from_pretrained(model_name,num_labels=2)
model_all.load_state_dict(torch.load(PRETRAINED_PATH))

<All keys matched successfully>

## Prepare Data

In [7]:
train_tok = preprocess_data(babe_train,tokenizer,'sentence')
test_tok = preprocess_data(babe_test,tokenizer,'sentence')
babe_tok = preprocess_data(babe_all,tokenizer,'sentence')

  0%|          | 0/4 [00:00<?, ?ba/s]

# Training

In [5]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=32,  
    logging_steps=20,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=3e-5)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
model.load_state_dict(torch.load(PRETRAINED_PATH))

model.to(device)
trainer = Trainer(model,training_args,train_dataset=train_tok,data_collator=data_collator
                      ,tokenizer=tokenizer)
trainer.train()

eval_dataloader = DataLoader(test_tok, batch_size=BATCH_SIZE, collate_fn=data_collator)
compute_metrics(model,device,eval_dataloader)['f1']

Step,Training Loss
20,0.561600
40,0.499300
60,0.507400
80,0.513800
100,0.438000
120,0.366600
140,0.305300
160,0.275800
180,0.290400
200,0.301900


0.8039869298268728

# Final model

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
model.load_state_dict(torch.load(PRETRAINED_PATH))

model.to(device)
trainer = Trainer(model,training_args,train_dataset=babe_tok,data_collator=data_collator
                      ,tokenizer=tokenizer)
trainer.train()

Step,Training Loss
20,0.564800
40,0.496300
60,0.461800
80,0.492600
100,0.448900
120,0.453500
140,0.308200
160,0.329600
180,0.297000
200,0.294000


0.9678364213527204

In [39]:
torch.save(model.state_dict(),FINAL_PATH)

In [40]:
model1 = BertForSequenceClassification.from_pretrained(model_name,num_labels=2)
model1.load_state_dict(torch.load(PRETRAINED_PATH))

<All keys matched successfully>

In [41]:
model2 = BertForSequenceClassification.from_pretrained(model_name,num_labels=2)
model2.load_state_dict(torch.load(FINAL_PATH))

<All keys matched successfully>

In [74]:
model3 = BertForSequenceClassification.from_pretrained(model_name,num_labels=2)
model3.load_state_dict(torch.load(SUBJ_PATH))

<All keys matched successfully>

In [75]:
model=model3

## Experiments

In [11]:
def classify_sentence(sent:str):
    toksentence = tokenizer(sent,truncation=True,return_tensors="pt")
    model.eval()
    model.to(device)
    with torch.no_grad():
        toksentence.to(device)
        output = model(**toksentence)
    
    classification = F.softmax(output.logits,dim=1).argmax(dim=1)
    
    return {0:'unbiased',1:'biased'}[classification[0].item()]

In [17]:
def classify_sentence_certainty(sent:str):
    toksentence = tokenizer(sent,truncation=True,return_tensors="pt")
    model.eval()
    model.to(device)
    with torch.no_grad():
        toksentence.to(device)
        output = model(**toksentence)
    
    classification = F.softmax(output.logits,dim=1)
    
    return classification

In [76]:
print(classify_sentence_certainty('Od března jedeme sedm dní v týdnu a bojujeme s touto ojedinělou situací'))
print(classify_sentence_certainty('ojedinělá situace, na kterou nemohl být nikdo připraven.'))

tensor([[0.7847, 0.2153]], device='cuda:0')
tensor([[0.0374, 0.9626]], device='cuda:0')


In [77]:
print(classify_sentence_certainty('Podle íránské vlády bylo sestřelení IR 655 "Vincennes" úmyslně provedeným a nezákonným činem.'))
print(classify_sentence_certainty('Sestřelení IR 655 "Vincennes" bylo úmyslně provedeným a nezákonným činem.'))

tensor([[0.1404, 0.8596]], device='cuda:0')
tensor([[0.4194, 0.5806]], device='cuda:0')


In [78]:
classify_sentence_certainty('Byli mu prý doporučeni jako "spolehliví a levní" a sám přesvědčil, že je droga od nich kvalitní a čistá.')

tensor([[0.3038, 0.6962]], device='cuda:0')

In [14]:
from newspaper import Article

In [15]:
from nltk import sent_tokenize

In [79]:
with open('text.txt','r') as f:
    text = f.read()


In [83]:
article = Article('https://www.seznamzpravy.cz/clanek/prectete-si-babisuv-projev-udelal-jsem-chybu-kterou-nechci-opakovat-120741')
article.download()
article.parse()
sentences = sent_tokenize(article.text)
annotations = np.array(list(zip(sentences,list(map(classify_sentence,sentences)))))
certainity = np.array(list(zip(sentences,list(map(classify_sentence_certainty,sentences)))))

stats = np.unique(annotations[:,1],return_counts=True)
print("bias level: ",stats[1][0]/stats[1].sum()*100,"%")

bias level:  75.30864197530865 %


In [84]:
print(certainity)

[['Vážení spoluobčané,\n\npo hezkém létě začíná podzim a my se opět nacházíme ve velmi složité situaci.'
  tensor([[0.3852, 0.6148]], device='cuda:0')]
 ['Budu k vám jako vždycky upřímný a otevřený.'
  tensor([[0.0551, 0.9449]], device='cuda:0')]
 ['Epidemie je zpět.' tensor([[0.4962, 0.5038]], device='cuda:0')]
 ['Bohužel.' tensor([[0.4712, 0.5288]], device='cuda:0')]
 ['Čísla nákazy závratně rostou.'
  tensor([[0.8814, 0.1186]], device='cuda:0')]
 ['V posledních dnech nám byly předložené scénáře vývoje, které mě značně znepokojily.'
  tensor([[0.4169, 0.5831]], device='cuda:0')]
 ['Bylo to podobné, jako když nám začátkem března analytici ukázali matematické modely, kterým bylo složité uvěřit, ale po třech dnech, když vycházely, jsme rázně udělali zásadní opatření.'
  tensor([[0.2590, 0.7410]], device='cuda:0')]
 ['Zachránili jsme tím tisíce životů, proto jsme první vlnu zvládli tak dobře.'
  tensor([[0.2333, 0.7667]], device='cuda:0')]
 ['Ke konci jara jsme ale měli pocit, že vir ode

In [86]:
list(filter(lambda x : x[1] == 'unbiased',annotations))

[array(['Čísla nákazy závratně rostou.', 'unbiased'], dtype='<U212'),
 array(['Bylo to poté, co jsme zarazili šíření nemoci jako jedni z prvních v Evropě.',
        'unbiased'], dtype='<U212'),
 array(['Od března jedeme sedm dní v týdnu a bojujeme s touto ojedinělou situací, na kterou nemohl být nikdo připraven.',
        'unbiased'], dtype='<U212'),
 array(['Sám jsem viděl případy, hlavně v Praze, kdy lidi nerespektovali ani ty úplně základní pokyny.',
        'unbiased'], dtype='<U212'),
 array(['Téměř každý je teď epidemiolog.', 'unbiased'], dtype='<U212'),
 array(['Jedenáctkrát více než u nás.', 'unbiased'], dtype='<U212'),
 array(['Také tam mají zakázané všechny kulturní akce nad 50 lidí.',
        'unbiased'], dtype='<U212'),
 array(['Proto si dovolím zopakovat základní pravidlo boje proti viru 3R: ruce, roušky, rozestupy.',
        'unbiased'], dtype='<U212'),
 array(['Na velké i menší skupiny lidí, zaměstnance, živnostníky i zaměstnavatele.',
        'unbiased'], dtype='<U212')

In [66]:
print("Začátky osídlovaní Horehroní se datují někdy okolo roku 2500 před naším letopočtem. Horský drsný terén nepředstavoval takové lákadlo jako níže položené oblasti a tehdejší obyvatelstvo potrápil při chovu dobytka a zemědělské činnosti.\nZlaté časy přišly až později ve 13. století. A to doslova zlaté. Velká část Horehroní byla totiž bohatá na zlato, které se v místních kopcích těžilo ve velkém. Například tehdejší obec Mýto pod Ďumbierom představovala jednu velkou továrnu o rozloze 600 hektarů, v níž se zpracovávala vytěžená hmota, a oddělovalo se zlato od hlušiny. Poprvé na světě byl pro tento proces těžby použit mlýn právě zde, díky čemuž je dodnes možné spatřit hory hlušiny, ale i pozůstatky štol v okolních horách.\nHorehroní se nachází na historicky slavné obchodní cestě Via Magna. Za účelem ochrany této trasy byl postaven hrad Ľupča, který je dnes možné navštívit. Dalšími hrady v regionu jsou Tisovský hrad a hrad Muráň, z nějž však zůstalo již jen torzo. Přesto však představuje oblíbené místo turistů a milovníků koní, zejména pro své unikátní vyhlídky na okolní přírodu Muránské planiny.\nSkrýval se zde Janošík\nTak jako bylo panstvo součástí historie Horehroní, tak jimi byli také zbojníci, kteří někdy bohatým brali a chudým dávali a někdy jen brali a nikomu nic nedali. V sedle Zbojská \"úřadoval\" zbojník Jakub Surovec, kterému je věnován naučný chodník při Salaši Zbojská.\nA o pár kopců dál, nad obcí Klenovec v roce 1713 chytli Janošíka, který se skrýval u Martina Mravce, taktéž zbojníka. Oba byli odsouzeni a krátce na to popraveni. V každém případě i tyto osobnosti vešly do dějin regionu, i když pouze jako opozice tehdejší moci. Zůstaly po nich však, pro turisty oblíbené, Zbojnické chodníčky nedaleko Salaše Zbojská.\nVelkým lákadlem také železnice\nDo dnešních dob na nás dýchá i historie technické revoluce. Nejznámějším parním vlakem na Slovensku je Čiernohronská železnice v Čiernom Balogu. K jejímu znovuzrození výraznou měrou přispěl původně dobrovolný studentský brigádník a současný ředitel, rodák z Plzně, Aleš Bílek.\nPod jeho vedením vlaky jezdí celoročně, dokonce i v tuhých zimách, a představují lákadlo, jak pro malé děti, tak pro dospělé, jelikož jejich kouzlo je nezapomenutelné. Tratě železnice vedou dolinami Slovenského rudohoří a jednou z jejich zastávek je i Lesnický skanzen, který své návštěvníky vtáhne do tajů života lesa, těžby dřeva a nabírání nových znalostí o přírodě.\nPřed pár lety byla také dobrovolníky z dezolátního stavu oživena ozubnicová železnice. Tento světový unikát dnes jezdí z Tisovce k Salaši Zbojská, na trati vinoucí se viadukty ve výšce 30 metrů a na úseku 4,7 kilometrů, Bánovo - Zbojská, překoná neuvěřitelné převýšení 166 metrů. Oba parní vlaky mají svá muzea v Čiernom Balogu a Tisovci.\nCelá řada muzeí\nZ monumentů minulosti, dávné i ne tak dávné, do dnešních dob zůstaly hrady, hradby, ale i muzea. Tím největším je určitě Horehronské muzeum v Brezně nacházející se ve třech budovách na náměstí. Zajímavá je i zrekonstruovaná synagoga se stálou expozicí holocaustu, ale i dechberoucí prostory, ve kterých probíhají umělecké vystoupení a vernisáže.\nNeméně zajímavým je Hutní muzeum v Podbrezové s vykopávkami z pradávných dob. Skvostem jsou i muzea: Zvonce na Šumiaci, muzeum miniatur v Pohorelej, ale také muzeum lyží a lanovek v Podkonici nebo Von Roll na svahu Chopku.")

Začátky osídlovaní Horehroní se datují někdy okolo roku 2500 před naším letopočtem. Horský drsný terén nepředstavoval takové lákadlo jako níže položené oblasti a tehdejší obyvatelstvo potrápil při chovu dobytka a zemědělské činnosti.
Zlaté časy přišly až později ve 13. století. A to doslova zlaté. Velká část Horehroní byla totiž bohatá na zlato, které se v místních kopcích těžilo ve velkém. Například tehdejší obec Mýto pod Ďumbierom představovala jednu velkou továrnu o rozloze 600 hektarů, v níž se zpracovávala vytěžená hmota, a oddělovalo se zlato od hlušiny. Poprvé na světě byl pro tento proces těžby použit mlýn právě zde, díky čemuž je dodnes možné spatřit hory hlušiny, ale i pozůstatky štol v okolních horách.
Horehroní se nachází na historicky slavné obchodní cestě Via Magna. Za účelem ochrany této trasy byl postaven hrad Ľupča, který je dnes možné navštívit. Dalšími hrady v regionu jsou Tisovský hrad a hrad Muráň, z nějž však zůstalo již jen torzo. Přesto však představuje oblíbené